<center><h1><b>Trabalho Computacional 01 </b></h1></center>

Autor: Rômulo Bandeira Pimentel Drumond

<h2>1. Questão 01</h2>

Na Questão 01 foi apresentado o conjunto de dados da Figura `\ref{fig:datasetForbes}`, nele consta medidas de pressão atmosférica e temperatura de ebulição da água para diferentes níveis de altitude. 

Figura: Conjunto de dados do experimento de Forbes
<img src="files/datasetForbes.png" style="width:60%;">
<small>Fonte: `\cite{TC1_ICA2018.pdf}`</small>

O primeiro passo realizado para a análise dos dados foi, devido a sua natureza bidimensional, plotar a relação entre temperatura de ebulição e pressão atmosférica. O código utilizado se encontra abaixo e o resultado na Figura `\ref{fig:datasetPlotly}`.

In [1]:
# Dados do fenômeno
te = {
    'valores': [194.5, 194.3, 197.9, 198.4, 199.4, 199.9, 200.9, 201.1,
              201.4, 201.3, 203.6, 204.6, 209.5, 208.6, 210.7, 211.9, 212.2],
    'nome': "Temperatura de ebulição (°F)"
}

p = {
    'valores': [20.79, 20.79, 22.40, 22.67, 23.15, 23.35, 23.89, 23.99, 
                24.02, 24.01, 25.14, 26.57, 28.49, 27.76, 29.04, 29.88, 30.06],
    'nome': "Pressão (polegadas de Hg)"
}

In [2]:
# Plot dos dados
import plotly.offline as plt
import plotly.graph_objs as go
plt.init_notebook_mode(connected=True) # habilitando plot dentro do jupyter notebook

def plotData(x, y):
    traceData = go.Scatter(
        x = x['valores'], 
        y = y['valores'], 
        mode='markers',
        name='dataset')
    data = [traceData]
    
    # fonte do texto do eixos da figura
    titlefont=dict(family='Courier New, monospace', size=18, color='#7f7f7f') 
    layoutData = go.Layout(
        title = "datasetPlotly",
        xaxis=dict(title=x['nome'], titlefont=titlefont),
        yaxis=dict(title=y['nome'], titlefont=titlefont)
    )

    fig = go.Figure(data=data, layout=layoutData)
    plt.iplot(fig)

x = te
y = p
plotData(x, y)

De forma a facilitar a compreensão deste trabalho a sequência de itens a, b, c e d proposta no documento `\cite{TC1_ICA2018.pdf}` não foi seguida a risca, em vez disso, o que foi solicitado foi enumerado abaixo e o estudo teve sua sequência baseado nele.

* Formulação do modelo estatístico e estimação dos parâmetros;
* Validação do modelo:
  * Estimação da variância do ruído a partir dos resíduos;
  * Distribuição dos resíduos via coeficiente de determinação (R2), histograma e boxplot;
  * Análise de outliers;
  * Análise de aceitação do modelo proposto.
* Reaplicação da metodologia de estudo na análise da dependência do logaritmo da pressão em relação ao ponto de ebulição;
* Análise comparativa entre o modelo original e o modelo com o logaritmo da pressão.

<h3><b>
1.1 Formulação do modelo estatístico e estimação de parâmetros
</b></h3>

O modelo mais simples que aparenta se adequar ao conjunto de dados é o linear, cuja forma é:

$$y=ax+b$$

Na equação acima $y$ é a variável dependente, visto que seu valor depende de $x$, que é a variável independente. Já as constantes $a$ e $b$ são os parâmetros do modelo linear. Para o problema proposto a pressão foi considerada a variável dependente ($y=p$) e a temperatura de ebulição a independente ($x=t_e$). Tratando-se de uma regressão ainda há uma variável extra: $\epsilon$, que representa o erro aleatório normalmente associado a ruído de medição. Reescrevendo a equação temos:

$$p=at_e+b+\epsilon$$

Algumas suposições desse modelo são que os erros possuem distribuição normal com média zero e variância $\sigma^2$ e são estatísticamente independentes (não há correlação entre erros) `\cite{Fundamentos de Regressão Linear.pdf}`.

A escolha entre quem seria a variável dependente e independente do modelo se deu porque Forbes, através de seus experimentos, pretendia estimar a altitude, que possui relação com a pressão, após medir a temperatura de ebulição da água.

Após adotado o modelo linear restou a estimação dos parâmetros do mesmo. Sabendo que temos $n$ amostras podemos representar de forma mais compacta, usando vetores e matrizes, a equação:

<center>$\vec{x}=[x_1, x_2, \dots , x_n]^t$</center>

<center>$\vec{y}=[y_1, y_2, \dots , y_n]^t$</center>

Onde $(x_i,y_i)$ é o ponto que representa a temperatura de ebulição com sua respectiva pressão atmosférica. Continuando o raciocínio:

<center>$\vec{y}=a\vec{x}+b\vec{1}_{nx1}+\vec{e}$</center>

Onde $\vec{e}$ representa o vetor de erros cuja dimensão é $n$.

<center>$\vec{y}=\big[\vec{x}, \vec{1}_{nx1}\big] \begin{bmatrix}a \\ b\end{bmatrix} +\vec{e}$</center>

Mudando o nome das variáveis, temos:

<center>$\vec{y}_{\text{nx1}}=A_{\text{nx2}} \vec{w}_{\text{nx1}} + \vec{e}_{\text{nx1}}$</center>

Supondo que o erro seja normal, aditivo e não-correlacionado, podemos estimar o vetor de parâmetros $\vec{w}$ usando Mínimos Quadrados Ordinários, ou *Ordinary Least Squares* (OLS). Portanto, a solução da última equação pode ser expressa em termos da pseudo-inversa da matriz $A$ `\cite{boyd}`:

<center>$\vec{w}=A^{\dagger} \vec{y}$</center>

O código abaixo foi utilizado para estimar os parâmetros. Como comparativo foi aplicado também as funções prontas da biblioteca *scikit-learn* para averiguar a similaridade dos métodos numéricos.

In [3]:
# Ordinary Least Squares
import numpy as np
from numpy.linalg import pinv

def ols(x, y):
    n = len(x['valores']) # tamanho do conjunto de dados
    
    # Aw = y
    ones = [1 for _ in range(0,n)]
    A = np.array([x['valores'], ones]).transpose()
    
    # pseudo-inverse of A
    A_cross = pinv(A)

    # w = A_cross*y
    w = np.matmul(A_cross, y['valores'])
    
    return w

w = ols(x, y)
print("Os parâmetros ótimos do modelo linear são: \na  = {}\nb = {}".format(w[0], w[1]))

Os parâmetros ótimos do modelo linear são: 
a  = 0.5228924007846376
b = -81.0637271286563


In [4]:
# Comparativo com scikit-learn:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit (np.array(x['valores']).reshape(-1,1), np.array(y['valores']).reshape(-1, 1))
print("Resultados com scikit-learn:")
print("a =", reg.coef_[0][0])
print("b =", reg.intercept_[0])

Resultados com scikit-learn:
a = 0.5228924007846357
b = -81.06372712865588


<h3><b>
1.2 Validação do modelo
</b></h3>

Para visualização do conjunto de dados, da regressão e dos erros a Figura `\ref{fig:regressao01}` foi gerada utilizando o código abaixo:

In [5]:
# Cálculo do erro para cada medida:
def errorVector(x,y,w):
    y_predicted = [w[0]*x + w[1] for x in x['valores']]
    e_vec = np.subtract(y['valores'], y_predicted)
    return e_vec
    
    
def plotRegression(x, y, w):
    # Gráfico da regressão linear
    x_pc = np.linspace(min(x['valores']), max(x['valores']), 100) # valores de x para traçar a curva pseudo-contínua

    # valores de y para o modelo linear
    y_regression = [w[0]*x + w[1] for x in x_pc] 
    e_vec = errorVector(x,y,w)

    # Plotagem do gráfico
    traceData = go.Scatter(
        x = x['valores'], 
        y = y['valores'], 
        mode='markers',
        name='dataset'
    )
    
    traceRegression = go.Scatter(
        x = x_pc,
        y = y_regression,
        mode='lines',
        name='modelo linear',
        yaxis = 'y2'
    )

    traceError = go.Bar(
        x = x['valores'],
        y = np.absolute(e_vec),
        name = "módulo do erro",
        yaxis = 'y3'
    )

    data = [traceError, traceRegression, traceData]

    delta = max(y['valores']) - min(y['valores']) # delta y
    prct = 0.1 # porcentagem de gap para cima  para baixo
    y_range = [min(y['valores'])-prct*delta, max(y['valores'])+prct*delta]

    titlefont=dict(family='Courier New, monospace', size=18, color='#7f7f7f')
    layoutRegression = go.Layout(
        title='Gráfico do dataset, modelo linear e respectivo erro',
        legend=dict(orientation="h", y=-.05),
        xaxis=dict(title = x['nome'], titlefont=titlefont),
        yaxis=dict(
            title = y['nome'],
            titlefont = titlefont,
            range = y_range
        ),
        yaxis2=dict(
            visible = False,
            range = y_range,
            overlaying='y',
            side='left'
        ),
        yaxis3=dict(
            title='Módulo do erro',
            titlefont=titlefont,
            anchor='x',
            overlaying='y',
            side='right'
        )
    )
    fig = go.Figure(data=data, layout=layoutRegression)
    plt.iplot(fig)
    
plotRegression(x, y, w)

O modelo parece se adequar aos dados, apenas no amostra próxima a 205 °F podemos ver um erro discrepante, esse será discutido quando analisarmos adiante a presença de outliers.

A variância dos resíduos, ou erros, foi estimada utilizando a fórmula abaixo `\cite{wikipedia?}`:

$$Var(e)=\frac{1}{n-1}\sum_{i=1}^{n} (e_i-\bar{e})^2$$

Onde $\bar{e}$ representa a média dos erros e $e_i$ o erro da $i$-ésima amostra.

In [6]:
def variance(x_vec):
    x_mean = np.mean(x_vec)
    var = (1/(len(x_vec)-1))*sum([(x - x_mean)**2 for x in x_vec])
    return var

e_vec = errorVector(x,y,w)
print("Variância dos resíduos: {}".format(variance(e_vec)))

Variância dos resíduos: 0.050821438352653715


Posteriormente, o Coeficiente de Determinação, ou $R^2$, foi calculado utilizando a fórmula `\cite{[Tutorial] Regressão Polinomial no MatlabOctave.pdf}`:

$$R^2=1-\frac{\sum_{i=1}^{n} (y_i-\hat{y}_i)^2}{\sum_{i=1}^{n} (y_i-\bar{y})^2}$$

Onde $y_i$ é o valor da pressão para cada amostra, $\hat{y}_i$ é a pressão predita pelo modelo linear e $\bar{y}$ é a média dos valores de pressão do conjunto de dados.

O código abaixo foi utilizado para efetuar o cálculo de $R^2$, gerar o histograma (Figura `\ref{fig:histogram01}`) e o Boxplot (Figura `\ref{fig:boxplot01}`) dos resíduos:

In [7]:
# Função que calcula o R2
def r2(y, e_vec):
    y_mean = np.mean(y['valores'])
        
    SQ_e = np.dot(np.transpose(e_vec), e_vec)          # sum of squared errors
    S_yy = sum([(y - y_mean)**2 for y in y['valores']]) # sum of errors for naive predictor

    R2 = 1 - SQ_e/S_yy
    return R2


def plotHistogram(e_vec):
    # Histograma dos resíduos
    data = [go.Histogram(
        x=e_vec,
        histnorm="probability density",
        #xbins=dict(start=-1, end=1, size=0.1)
    )]
    
    titlefont=dict(family='Courier New, monospace', size=18, color='#7f7f7f')
    layoutHist = go.Layout(
        title='Histograma dos resíduos',
        xaxis=dict(title='Valor do erro', titlefont=titlefont),
        yaxis=dict(title='Densidade de probabilidade', titlefont=titlefont)
    )

    fig = go.Figure(data=data, layout=layoutHist)
    plt.iplot(fig)

# Boxplot dos resíduos
def plotBoxplot(e_vec):
    data = [go.Box(
        y= e_vec,
        boxpoints='all',
        boxmean='sd',
        jitter=0.3,
        pointpos=-1.8
    )]
    
    layoutBoxplot = go.Layout(title="Boxplot dos resíduos")

    fig = go.Figure(data=data, layout=layoutBoxplot)
    plt.iplot(fig)

error_vec = errorVector(x,y,w) # vetor dos erros entre as medições e o modelo linear

R2 = r2(y, error_vec)
print("R2 = {}".format(R2))

plotHistogram(error_vec)
plotBoxplot(error_vec)

R2 = 0.9944281526462319


Como o valor de $R^2$ foi alto podemos dizer que o modelo aparenta se adequar bem às amostras. O histograma não aparenta, visualmente, seguir uma distribuição gaussiana, principalmente pela falta de simetria da distribuição. Um indício dessa falta de normalidade está contido no boxplot dos resíduos, que apresenta a existência de um outlier. Para ter mais rigor na análise da hipótese de normalidade dos resíduos foi aplicado o teste de Shapiro-Wilk de normalidade com 95% de confiança e 5% de significância:

In [8]:
from scipy import stats
print("P-value: {}".format(stats.shapiro(error_vec)[1]))

P-value: 0.02394944056868553


Visto que o resultado do teste apresentou p-value inferior a 0,05 foi rejeitada a hipótese que a distribuição dos resíduos é normal `\cite{refShapiro-Wilk}`, então, apesar do valor de $R^2$ ser alto uma das suposições do modelo linear não foi atendida.

Nos faltou analisar a presença de outliers. Nas Figuras `\ref{fig:regressao01}` e  `\ref{fig:boxplot01}` podemos ver que há uma amostra com erro discrepante, o outlier. O código abaixo foi utilizado para identifica-lo, apresentar suas coordenadas, valor do erro e distância da média em número de desvios padrões:

In [9]:
index = np.argmax(np.absolute(error_vec))  # índice do outlier
erro_outlier = max(np.absolute(error_vec)) # valor do erro do outlier
distancia = (erro_outlier - np.mean(error_vec)) / np.sqrt(variance(error_vec))

print("Outlier:\nPonto: ({}, {}) \nValor do erro: {} \nDistância: {} desvios-padrões"
      .format(x['valores'][index], y['valores'][index], erro_outlier, distancia))

Outlier:
Ponto: (204.6, 26.57) 
Valor do erro: 0.6499419281194392 
Distância: 2.8830427240926944 desvios-padrões


De forma a sintetizar os resultados a Tabela `\ref{tab:resultados01}` foi elaborada.

In [16]:
import pandas as pd
dfB = pd.DataFrame({"Parâmetros da regressão": [w],
                    "R²": R2,
                    "Var(e)": variance(error_vec),
                    "SW P-value": stats.shapiro(error_vec)[1]
                  })
dfB

,Parâmetros da regressão,R²,Var(e),SW P-value
0,"[0.5228924007846376, -81.0637271286563]",0.994428,0.050821,0.023949


Para analisar o efeito do outlier na regressão linear a amostra foi retirada e o mesmo estudo foi refeito.

In [17]:
# Criando função para fazer a análise completa da regressão linear
def regressionAnalysis(x, y):
    # x = {'valores': [data], 'nome': 'dimensão-dos-dados'}
    # y = {'valores': [data], 'nome': 'dimensão-dos-dados'}
            
    w = ols(x,y) # Regressão linear
    
    error_vec = errorVector(x,y,w) # vetor dos erros
    R2 = r2(y, error_vec)          # cálculo do R2
    
    plotRegression(x, y, w)  # gráfico da regressão linear
    plotHistogram(error_vec) # histograma dos resíduos
    plotBoxplot(error_vec)   # boxplot dos resíduos
    
    # resultados organizados num panda dataframe
    return pd.DataFrame({"Parâmetros da regressão": [w],
                         "R²": R2,
                         "Var(e)": variance(error_vec),
                         "SW P-value": stats.shapiro(error_vec)[1]
                        })

In [18]:
# Limpando o conjunto de dados
x_clean = x.copy()
x_clean['valores'] = [q for q in x['valores'] if q != x['valores'][index]]
y_clean = y.copy()
y_clean['valores'] = [q for q in y['valores'] if q != y['valores'][index]]

# Nova análise
dfB2 = regressionAnalysis(x_clean, y_clean)
dfB2

,Parâmetros da regressão,R²,Var(e),SW P-value
0,"[0.520737829134305, -80.66729363989373]",0.997478,0.024124,0.366088


Os resultados das duas regressões podem ser vistos na `\ref{tab:compReg01e02}`.

In [19]:
compReg01e02 = pd.concat([dfB, dfB2])
compReg01e02.index = ["Com outlier", "Sem outlier"]
compReg01e02

,Parâmetros da regressão,R²,Var(e),SW P-value
Com outlier,"[0.5228924007846376, -81.0637271286563]",0.994428,0.050821,0.023949
Sem outlier,"[0.520737829134305, -80.66729363989373]",0.997478,0.024124,0.366088


Podemos ver que a retirada do outlier melhorou as métricas utilizadas para avaliar o modelo: 
  * Valor de $R^2$ aumentou;
  * Variância dos resíduos diminuiu;
  * P-value do teste de Shapiro-Wilk aumentou ao ponto de permitir a aceitação da hipótese de normalidade dos resíduos.

Apesar do aumento de performance do modelo não é indicado a retirada do outlier devido ao tamanho do conjunto de dados (apenas 17 amostras). Uma atitude mais coerente seria voltar ao campo e obter mais dados, dessa forma o efeito dos outliers seria diluído a medida que o \textit{dataset} aumentasse.

<h3><b>
1.3 Análise da dependência entre o logaritmo da pressão e o ponto de ebulição
</b></h3>

Inicialmente um gráfico do Logaritmo da Pressão versus Temperatura de Ebulição foi feito, o resultado pode ser visto na Figura `\ref{fig:dataset02}`.

In [20]:
from math import log

logP = {
    'valores': [log(p) for p in p['valores']], # Aplicando a função logarítimo nos valores da pressão
    'nome':   "Logarítimo da "+ p['nome']
}

y = logP
x = te

plotData(x,y)

O modelo linear foi adotado novamente, consequentemente assumindo a seguinte forma:

$$log(p)=at_e+b+\epsilon$$

A estimação dos parâmetros foi feita utlizando o Método dos Mínimos Quadrados Ordinários e o resultado da análise se encontra a seguir.

In [21]:
dfD = regressionAnalysis(x, y)
dfD

,Parâmetros da regressão,R²,Var(e),SW P-value
0,"[0.020622361139063856, -0.9708662096193041]",0.994961,0.000071,0.000003


Analogamente ao modelo linear do `tópico 1.2` a regressão apresentou:
  * Alto valor de $R^2$;
  * Rejeição da hipótese de normalidade a partir do teste de Shapiro-Wilk.

Comparando os dois modelos:

In [23]:
compBeD = pd.concat([dfB, dfD])
compBeD.index = ["Modelo original", "Modelo com log(p)"]
compBeD

,Parâmetros da regressão,R²,Var(e),SW P-value
Modelo original,"[0.5228924007846376, -81.0637271286563]",0.994428,0.050821,0.023949
Modelo com log(p),"[0.020622361139063856, -0.9708662096193041]",0.994961,0.000071,0.000003


Percebe-se que o modelo logarítmico apresentou um valor de $R^2$ levemente mais alto, mas igualmente teve a hipótese de normalidade dos resíduos rejeitada. Apesar da variância dos resíduos apresentar dessa vez um valor muito baixo não é sensato compará-lo com o modelo anterior devido à mudança de escala das medidas, com pressões agora em escala logarítmica.

Novamente o outlier foi identificado, retirado e o estudo refeito.

In [24]:
error_vec = errorVector(x,y,ols(x,y))      # vetor de erros
index = np.argmax(np.absolute(error_vec))  # índice do outlier
erro_outlier = max(np.absolute(error_vec)) # valor do erro do outlier
distancia = (erro_outlier - np.mean(error_vec)) / np.sqrt(variance(error_vec))

print("Outlier:\nPonto: ({}, {}) \nValor do erro: {} \nDistância: {} desvios-padrões"
      .format(x['valores'][index], y['valores'][index], erro_outlier, distancia))

Outlier:
Ponto: (204.6, 3.279782759771722) 
Valor do erro: 0.031313880338561084 
Distância: 3.7043665949927687 desvios-padrões


In [25]:
# Limpando dataset
x_clean = x.copy()
x_clean['valores'] = [q for q in x['valores'] if q != x['valores'][index]]
y_clean = y.copy()
y_clean['valores'] = [q for q in y['valores'] if q != y['valores'][index]]

# New analysis
dfD2 = regressionAnalysis(x_clean, y_clean)
dfD2

,Parâmetros da regressão,R²,Var(e),SW P-value
0,"[0.020518554943586467, -0.9517662403878724]",0.999569,0.000006,0.232378


Comparando os dois resultados do tópico atual:

In [27]:
compDeD2 = pd.concat([dfD, dfD2])
compDeD2.index = ["Modelo log(p) com outlier", "Modelo log(p) sem outlier"]
compDeD2

,Parâmetros da regressão,R²,Var(e),SW P-value
Modelo log(p) com outlier,"[0.020622361139063856, -0.9708662096193041]",0.994961,0.000071,0.000003
Modelo log(p) sem outlier,"[0.020518554943586467, -0.9517662403878724]",0.999569,0.000006,0.232378


Podemos enumerar os efeitos positivos da retirada do outlier nas métricas adotadas para validar o modelo:
  * Valor de $R^2$ aumentou;
  * Variância dos resíduos diminuiu;
  * P-value do teste de Shapiro-Wilk aumentou ao ponto de permitir a aceitação da hipótese de normalidade dos resíduos.
  
Os resultados foram parecidos com o da seção anterior: o outlier afetou a normalidade dos resíduos, mas, a melhor forma de proceder é adquirir mais dados em vez de produzir um modelo retirando o outlier do *dataset*.

Finalmente, comparou-se os quatro modelos:

In [28]:
resultado = pd.concat([dfB, dfB2, dfD, dfD2])
resultado.index = ["Modelo original com outlier", 
                   "Modelo original sem outlier", 
                   "Modelo log(p) com outlier",  
                   "Modelo log(p) sem outlier"]
resultado

,Parâmetros da regressão,R²,Var(e),SW P-value
Modelo original com outlier,"[0.5228924007846376, -81.0637271286563]",0.994428,0.050821,0.023949
Modelo original sem outlier,"[0.520737829134305, -80.66729363989373]",0.997478,0.024124,0.366088
Modelo log(p) com outlier,"[0.020622361139063856, -0.9708662096193041]",0.994961,0.000071,0.000003
Modelo log(p) sem outlier,"[0.020518554943586467, -0.9517662403878724]",0.999569,0.000006,0.232378


Conclui-se que o outlier afetou negativamente os modelos nos dois casos e que de forma geral o modelo logarítmico (\textit{i.e.} modelo linear utilizando logarítmo da pressão como variável dependente) apresentou o valor de $R^2$ levemente mais alto que o modelo linear original.